In [27]:
import pandas as pd

In [2]:
ma = pd.read_pickle('data/memento_avail.pkl.gz', compression='gzip')

In [3]:
mad = [({'original_uri': m['original_uri']}, {x['archive_id']: True for x in m['timemap_index']}) for m in ma.tolist() if m is not None]

In [4]:
mad = [{**x, **y} for x, y in mad]

In [5]:
mad = pd.DataFrame(mad).fillna(False).set_index('original_uri')

In [6]:
(mad.sum(axis=0)/127200).sort_values(ascending=False)

es                    0.895055
cat                   0.895055
webcite               0.895055
ia                    0.895055
si                    0.895055
aueb                  0.895055
is                    0.895055
eot                   0.895055
nrs                   0.895055
archiveit             0.700810
gcwa                  0.638624
uknationalarchives    0.621596
proni                 0.577940
archive.is            0.518836
swa                   0.509167
nli                   0.496305
loc                   0.470503
blarchive             0.448333
pt                    0.373263
ba                    0.165346
hr                    0.111612
perma                 0.096588
bsb                   0.015896
wiki                  0.014764
nara                  0.003341
yorku                 0.000094
git                   0.000031
wikia                 0.000024
archive.li            0.000016
archive.fo            0.000016
cs.dbpedia.org        0.000008
fb.me                 0.000008
archive.

In [7]:
wb = pd.read_pickle('data/wb_avail_all_sources_df_FINAL.pkl.gz', compression='gzip')

In [8]:
wb[wb.available==False].uri.iat[0]

'orientaldaily.com.my/index.php?option=com_k2&view=item&id=131282:&Itemid=197'

## CDX

In [9]:
import requests
from functools import wraps

def retry(times):

    def wrapper_fn(f):

        @wraps(f)
        def new_wrapper(*args,**kwargs):
            for i in range(times):
                try:
                    #print 'try %s' % (i + 1)
                    return f(*args,**kwargs)
                except Exception as e:
                    error = e
            raise error

        return new_wrapper

    return wrapper_fn


@retry(3)
def get_cdx_avail(base_uri, uri):
    try:
        resp = requests.get(base_uri, params={'url': uri})
        resp = (resp.url, resp.status_code, resp.text)
    except:
        resp = None
    return resp

In [25]:
base_uri = 'http://web.archive.org/cdx/search/cdx'
x = get_cdx_avail(base_uri, wb[wb.available==False].uri.iat[0]+'2139023')

In [26]:
x[2]

''

In [28]:
df = pd.read_pickle('lang_uri_time_all.pkl.gz', compression='gzip')
df.head()

,lang,source,timestamp,uri,uri_cln,tld,is_shortened,uri_unshrtn,uri_cln_unshrtn,uri_unshrtn_bitly,uri_unshrtn_raw,tld_unshrtn
0,zh,wikipedia,2018-08-22 13:21:14,https://web.archive.org/web/20150626164255/htt...,web.archive.org/web/20150626164255/http://www....,archive.org,False,NaN,web.archive.org/web/20150626164255/http://www....,NaN,NaN,archive.org
1,zh,wikipedia,2018-06-25 15:15:42,https://web.archive.org/web/20140727005451/htt...,web.archive.org/web/20140727005451/http://trav...,archive.org,False,NaN,web.archive.org/web/20140727005451/http://trav...,NaN,NaN,archive.org
2,zh,wikipedia,2017-11-10 21:53:43,https://archive.is/20140828023420/http://www.w...,archive.is/20140828023420/http://www.worldjour...,archive.is,False,NaN,archive.is/20140828023420/http://www.worldjour...,NaN,NaN,archive.is
3,zh,wikipedia,2017-11-10 21:53:43,https://web.archive.org/web/20140803200140/htt...,web.archive.org/web/20140803200140/http://www....,archive.org,False,NaN,web.archive.org/web/20140803200140/http://www....,NaN,NaN,archive.org
4,zh,wikipedia,2017-11-10 21:53:43,https://web.archive.org/web/20140808044248/htt...,web.archive.org/web/20140808044248/http://m.al...,archive.org,False,NaN,web.archive.org/web/20140808044248/http://m.al...,NaN,NaN,archive.org


In [29]:
unique_uris = pd.concat([df.uri_cln, df.uri_cln_unshrtn]).drop_duplicates()
unique_uris = unique_uris[~(unique_uris.str.contains('archive.org').fillna(False))].to_frame().rename(columns={0: 'uri'})
unique_uris.head()

,uri
2,archive.is/20140828023420/http://www.worldjour...
8,cn.rfi.fr/%E6%94%BF%E6%B2%BB/20140720-%E8%81%9...
9,nymag.com/daily/intelligencer/2014/07/hamas-di...
10,jpost.com/Operation-Protective-Edge/50-days-of...
11,terrorism-info.org.il/Data/articles/Art_20774/...


In [118]:
from tqdm import tqdm

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm_gui`, `tqdm_notebook`, optional kwargs, etc.)
tqdm.pandas(desc="my bar!")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
#df.progress_apply(lambda x: x**2)
# can also groupby:
# df.groupby(0).progress_apply(lambda x: x**2)

In [30]:
cdx_avail = unique_uris.progress_apply(lambda x: get_cdx_avail(base_uri, x.uri), axis=1)

my bar!:  19%|█▉        | 23408/124286 [2:58:37<10:54:41,  2.57it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

my bar!:  24%|██▍       | 30136/124286 [3:48:29<10:01:41,  2.61it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

my bar!:  30%|██▉       | 36902/124286 [4:43:18<14:04:08,  1.73it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change th

In [32]:
cdx_avail.head()

2     (http://web.archive.org/cdx/search/cdx?url=arc...
8     (http://web.archive.org/cdx/search/cdx?url=cn....
9     (http://web.archive.org/cdx/search/cdx?url=nym...
10    (http://web.archive.org/cdx/search/cdx?url=jpo...
11    (http://web.archive.org/cdx/search/cdx?url=ter...
dtype: object

In [25]:
cdx_avail = pd.read_pickle('data/cdx_avail.pkl.gz', compression='gzip')

In [26]:
cdx_avail.head()

2     (http://web.archive.org/cdx/search/cdx?url=arc...
8     (http://web.archive.org/cdx/search/cdx?url=cn....
9     (http://web.archive.org/cdx/search/cdx?url=nym...
10    (http://web.archive.org/cdx/search/cdx?url=jpo...
11    (http://web.archive.org/cdx/search/cdx?url=ter...
dtype: object

In [30]:
cdx_avail = pd.concat([unique_uris, cdx_avail], axis=1)

In [68]:
'2132\n'.strip().split('\n')

['2132']

In [69]:
cdx_status = pd.DataFrame([(u, x[0], x[1], len(x[2].strip().split('\n')))
                            if len(x[2])>0 else (u, x[0], x[1], 0)
                           for u, x in zip(cdx_avail.iloc[:, 0].tolist(), cdx_avail.iloc[:, 1])
                           ], 
                          columns=['uri', 'cdx_url', 'status', 'lines']) 

In [72]:
cdx_status.head()

,uri,cdx_url,status,lines
0,archive.is/20140828023420/http://www.worldjour...,http://web.archive.org/cdx/search/cdx?url=arch...,200,0
1,cn.rfi.fr/%E6%94%BF%E6%B2%BB/20140720-%E8%81%9...,http://web.archive.org/cdx/search/cdx?url=cn.r...,200,2
2,nymag.com/daily/intelligencer/2014/07/hamas-di...,http://web.archive.org/cdx/search/cdx?url=nyma...,200,154
3,jpost.com/Operation-Protective-Edge/50-days-of...,http://web.archive.org/cdx/search/cdx?url=jpos...,200,284
4,terrorism-info.org.il/Data/articles/Art_20774/...,http://web.archive.org/cdx/search/cdx?url=terr...,200,21


In [63]:
99636 / unique_uris.shape[0]

0.8016671226043158

In [64]:
cdx_status = pd.read_csv('data/cdx_avail_lines.csv.gz', compression='gzip')

In [75]:
cdx_status[cdx_status.lines==1].head()

,uri,cdx_url,status,lines
32,m.news.sina.com.tw/article/20140802/13028508.html,http://web.archive.org/cdx/search/cdx?url=m.ne...,200,1
69,nzherald.co.nz/world/news/article.cfm?c_id=2&o...,http://web.archive.org/cdx/search/cdx?url=nzhe...,200,1
71,tw.news.yahoo.com/%E4%BB%A5%E5%B7%B4%E8%A1%9D%...,http://web.archive.org/cdx/search/cdx?url=tw.n...,200,1
72,chinese.rfi.fr/%E6%94%BF%E6%B2%BB/20140720-%E8...,http://web.archive.org/cdx/search/cdx?url=chin...,200,1
82,haluan.org.my/v5/index.php/palestin/berita-pal...,http://web.archive.org/cdx/search/cdx?url=halu...,200,1


In [76]:
print(cdx_avail.reset_index()[cdx_status.lines==2].iat[0, 2][2])

fr,rfi,cn)/%e6%94%bf%e6%b2%bb/20140720-%e8%81%94%e5%90%88%e5%9b%bd%e7%a7%98%e4%b9%a6%e9%95%bf%e5%89%8d%e5%be%80%e4%b8%ad%e4%b8%9c%e6%96%a1%e6%97%8b%e5%b7%b4%e4%bb%a5%e5%86%b2%e7%aa%81 20190213020443 http://cn.rfi.fr/%E6%94%BF%E6%B2%BB/20140720-%E8%81%94%E5%90%88%E5%9B%BD%E7%A7%98%E4%B9%A6%E9%95%BF%E5%89%8D%E5%BE%80%E4%B8%AD%E4%B8%9C%E6%96%A1%E6%97%8B%E5%B7%B4%E4%BB%A5%E5%86%B2%E7%AA%81 text/html 200 JL77CB7N3XZD6IVUNB2PVY6CWFMVAG5D 29686
fr,rfi,cn)/%e6%94%bf%e6%b2%bb/20140720-%e8%81%94%e5%90%88%e5%9b%bd%e7%a7%98%e4%b9%a6%e9%95%bf%e5%89%8d%e5%be%80%e4%b8%ad%e4%b8%9c%e6%96%a1%e6%97%8b%e5%b7%b4%e4%bb%a5%e5%86%b2%e7%aa%81 20190214102140 http://cn.rfi.fr/%E6%94%BF%E6%B2%BB/20140720-%E8%81%94%E5%90%88%E5%9B%BD%E7%A7%98%E4%B9%A6%E9%95%BF%E5%89%8D%E5%BE%80%E4%B8%AD%E4%B8%9C%E6%96%A1%E6%97%8B%E5%B7%B4%E4%BB%A5%E5%86%B2%E7%AA%81 text/html 200 GRDBOLM2B4TSCKZSANGETZQ4AKJHH3J4 29971



In [77]:
cdx_status.lines.value_counts()

0         99636
1          4805
2          3140
3          2288
4          1744
5          1365
6           958
7           857
8           700
9           552
10          482
11          426
12          330
13          286
14          269
15          260
16          239
19          199
17          194
18          191
20          175
21          156
22          153
23          140
24          129
25          121
26          118
27          117
28          115
29           97
          ...  
1443          1
5537          1
931           1
867           1
483           1
355           1
4257          1
613           1
1381          1
360           1
359           1
210959        1
26603         1
3878          1
1767          1
1511          1
935           1
807           1
551           1
295           1
3748          1
15585         1
4836          1
4644          1
486           1
422           1
358           1
294           1
230           1
351           1
Name: lines, Length: 921

In [45]:
cdx_avail.iat[0]

('http://web.archive.org/cdx/search/cdx?url=archive.is%2F20140828023420%2Fhttp%3A%2F%2Fwww.worldjournal.com%2Fview%2Ffull_van%2F25637070%2Farticle-%25E5%2581%259C%25E7%2581%25AB%25E5%258F%2588%25E7%25A0%25B4%25E5%25B1%2580-%25E5%2593%2588%25E7%2591%25AA%25E6%2596%25AF%25E6%258C%2587%25E6%258F%25AE%25E5%25AE%2598%25E5%25A6%25BB%25E5%25A5%25B3%25E8%25A2%25AB%25E7%2582%25B8%25E6%25AD%25BB%3Finstance%3Dbc_bull_left1',
 200,
 '')

In [105]:
wb = cdx_status[['uri', 'lines']].rename(columns={'lines': 'cdx'}).merge(wb_df[['uri', 'available']], on='uri', how='left').fillna(False)

In [108]:
wb[(wb.cdx!=0) & (~wb.available)].shape

(2057, 3)

In [110]:
wb.head()

,uri,cdx,available
0,archive.is/20140828023420/http://www.worldjour...,0,False
1,cn.rfi.fr/%E6%94%BF%E6%B2%BB/20140720-%E8%81%9...,2,True
2,nymag.com/daily/intelligencer/2014/07/hamas-di...,154,True
3,jpost.com/Operation-Protective-Edge/50-days-of...,284,True
4,terrorism-info.org.il/Data/articles/Art_20774/...,21,True


In [13]:
@retry(3)
def get_memento_avail(base_uri, uri):
    try:
        resp = requests.get(base_uri+uri).json()
    except:
        resp = None
    return resp

In [16]:
base_uri_time =  'http://timetravel.mementoweb.org/api/json/20140801000000/'


In [17]:
get_memento_avail(base_uri_time, 'english.alarabiya.net/en/News/2014/07/30/Hamas-official-asks-Hezbollah-to-join-fight-against-Israel-.html')

In [ ]:
memento_api2_avail = unique_uris.progress_apply(lambda x: get_memento_avail(base_uri_time, x.uri), axis=1)

my bar!:   8%|▊         | 10560/124286 [16:11:39<1871:59:43, 59.26s/it]

In [ ]:
memento_api2_avail.head()

## Status code check

In [127]:
def retry_none(times):

    def wrapper_fn(f):

        @wraps(f)
        def new_wrapper(*args,**kwargs):
            for i in range(times):
                try:
                    #print 'try %s' % (i + 1)
                    return f(*args,**kwargs)
                except Exception as e:
                    error = e
            return None

        return new_wrapper

    return wrapper_fn

In [128]:
@retry_none(3)
def get_response(uri):
    resp = requests.get(uri)
    return resp

In [129]:
df['uri_unsh_no_clean'] = df.uri_unshrtn_raw.combine_first(df.uri_unshrtn_bitly.combine_first(df.uri_unshrtn.combine_first(df.uri)))

In [124]:
unique_uris_raw_only_unsh = df.uri_unsh_no_clean.drop_duplicates().to_frame().rename(columns={'uri_unsh_no_clean': 'uri'})
unique_uris_raw_only_unsh.shape

(83590, 1)

In [132]:
import concurrent.futures

In [ ]:
all_resp = {}

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    # Start the load operations and mark each future with its URL
    future_to_parse = {executor.submit(get_response, uri): uri for idx, uri in unique_uris_raw_only_unsh.itertuples()}
    for future in concurrent.futures.as_completed(future_to_parse):
        uri = future_to_parse[future]
        all_resp[uri] = future


for future in concurrent.futures.as_completed(future_to_parse):
    uri = future_to_parse[future]
    try:
        all_resp[uri] = future.result()
    except:
        all_resp[uri] = None

In [ ]:
len(all_resp)

In [5]:
import os; os.getpid()

13763

## 2014_NLIL

In [1]:
import pandas as pd


In [5]:
nl = (pd.read_csv('data/collscan-daan-nlil_2014.cdx.gz', sep=' ', header=None, names=['timestamp', 'url'], usecols=[1,2])
      .assign(timestamp=lambda x: x.timestamp.astype(int)))
nl.head()

,timestamp,url
0,20140731225812,http://9tv.co.il/
1,20140731230439,http://9tv.co.il/news/2014/07/31/181742.html
2,20140731230606,http://9tv.co.il/news/2014/07/31/181835.html
3,20140731230004,http://9tv.co.il/news/protective-edge/
4,20140731225747,http://85fashionkitty.blogspot.co.il/


In [7]:
nl.timestamp.agg([min, max])

min    20140731225138
max    20141021015238
Name: timestamp, dtype: int64